<a id='0'></a>
### 　　目　录
* [第1章 初探大数据](#1)
* [第2章 Spark及其生态圈概述](#2)
* [第3章 实战环境搭建](#3)

In [2]:
%autosave 3

Autosaving every 3 seconds


<a id='1'></a>
# [第1章 初探大数据](#0)


目录：/home/hadoop
software: 存放安装的软件
app:存放所有软件的安装目录
data：存放课程所有的测试数据目录
source：存放的是软件源码的目录，如spark

groupadd hadoop      创建hadoop用户组  
useradd -m -g hadoop hadoop    新建hadoop用户并增加到hadoop用户组中  

passwd hadoop     hadoop用户密码，为hadoop  

环境参数<img src = 'https://upload-images.jianshu.io/upload_images/6344527-77faf8b26e275559.png?imageMogr2/auto-orient/strip%7CimageView2/2/w/1240' width='66%'>

**software 中的软件**![s**oftware 中的软件**](https://upload-images.jianshu.io/upload_images/6344527-16dc3af5ae916884.png?imageMogr2/auto-orient/strip%7CimageView2/2/w/1240)

## 10 
### 1.hadoop 下载
wget http://archive.cloudera.com/cdh5/cdh/5/hadoop-2.6.0-cdh5.7.0.tar.gz  
tar -zxvf hadoop-2.6.0-cdh5.7.0.tar.gz   -C ~/app/

  
### jdk 下载
wget http://ftp.upf.br/pub/linux/java/jdk-7u51-linux-x64.tar.gz
### 解压
tar -zxvf jdk-7u51-linux-x64.tar.gz -CC ~/app/

验证安装

配置到系统**环境变量**

vi  ~/.bash_profile  
source ~/.bash_profile
```
export JAVA_HOME=/home/hadoop/app/jdk1.7.0_51
export PATH=$JAVA_HOME/bin:$PATH

export HADOOP_HOME=/home/hadoop/app/hadoop-2.6.0-cdh5.7.0
export PATH=$HADOOP_HOME/bin:$PATH

export HIVE_HOME=/home/hadoop/app/hive-1.1.0-cdh5.7.0  
export PATH=$HIVE_HOME/bin:$PATH  
```
### 3.机器参数设置
    hostname: hekuang
    修改：/etc/sysconfig/network
ubuntu: /etc/network/interfaces.

设置ip和hostname的映射关系：/etc/hosts


### ssh免密码登录 

ssh-keygen  -t rsa　　
cp ~/.ssh/id_rsa.pub ~/.ssh/authorized_keys  
再追加到各个主机实现，免密码登录

### hadoop配置文件修改
* bin : 客户端操作的文件
* sbin ： 启动集群  

~/app/hadoop-2.6.0-cdh5.7.0$ cd etc/hadoop

* 修改 hadoop-env.sh

> `export JAVA_HOME=/home/hadoop/app/jdk1.7.0_51`

*  修改 etc/hadoop/core-site.xml

```
 <configuration>  
    <property>
        <name>fs.defaultFS</name>
        <value>hdfs://207.148.26.255:8020</value>
    </property>

<property>
        <name>hadoop.tmp.dir</name>
        <value>/home/hadoop/app/tmp</value>
    </property>
</configuration>
```

* 修改 hdfs-site.xml
```
<property>
        <name>dfs.replication</name>
        <value>1</value>
    </property>
```
* vi slaves
改为本机ip

* 仅一次格式化 bin/hdfs namenode -format
* 启动 sbin/start-dfs.sh

## YARN 产生背景
* MapReduce 1.x 存在问题：单点故障，节点压力大不易扩展  
* JobTracker : 负责资源管理和作业调度
* TaskTracker : 

 YARN : 不同计算框架可以共享同一个HDFS集群上的数据，享受整体的资源调度   
 XXX on YARN   Yet Another Rsesource Negotiator

## YARN 架构
 YARN 架构 
[http://hadoop.apache.org/docs/current/hadoop-yarn/hadoop-yarn-site/YARN.html](http://hadoop.apache.org/docs/current/hadoop-yarn/hadoop-yarn-site/YARN.html)

* ResouceManager ：RM   
   整个集群同一时间提供服务的RM只有**一个**，负责集群资源的统一调度和管理  
  监控NM， 一旦NM挂掉，该NM运行的任务需要告诉我们的AM来如何进行 处理
  处理客户端请求：提交一个作业，杀手一个作业  

* etc/hadoop/mapred-site.xml:
cp mapred-site.xml.template mapred-site.xml

```
<configuration>
    <property>
        <name>mapreduce.framework.name</name>
        <value>yarn</value>
    </property>
</configuration>\
```

* etc/hadoop/yarn-site.xml:


```
<configuration>
    <property>
        <name>yarn.nodemanager.aux-services</name>
        <value>mapreduce_shuffle</value>
    </property>
</configuration>
```
* 启动   $ sbin/start-yarn.sh
* 验证 ResourceManager - http://localhost:8088/
*　提交Mr作业到YARN上运行  
hadoop jar  /home/hadoop/app/hadoop-2.6.0-cdh5.7.0/share/hadoop/mapreduce/hadoop-mapreduce-examples-2.6.0-cdh5.7.0.jar wordcount /input/wc/hello.txt /output/wc/

* 提交作业到yarn
date 中创建txt文件  
hadoop fs -mkdir -p /input/wc   
hadoop fs -put  hello.txt /input/wc/

## Hive
#### 1.21 Hive产生背景
* MapReduce 编程的不便性
* HDFS 的文件缺少Schema  
##### Hive 是什么  
* Facebook开源，解决海量结构化的日志数据统计问题
* 构建在Hadoop之上的数据仓库
* Hive 定义类一种SQL查询语言：HQL（类似SQL但不完全相同）
* 通常进行离线数据处理（采用MapReduce）
* 底层支持多种不同的执行格式
* 支持多种不同的压缩格式、存储格式以及自定义函数 压缩：GZIP、LZO、Snappy

#### 1.22 
为什么使用Hive
* 简单，易上手
* 为超大数据集设计的计算/存储扩展能力
* **统一的元数据管理（可与Presto/Impala/SparkSQL等共享数据）**

#### 1.23 大数据数据仓库Hive架构以及部署
### [Hive安装配置](http://blog.51cto.com/hatech/1427748)
#### 1-24 -Hive环境搭建
* 下载：
   * wget http://archive.cloudera.com/cdh5/cdh/5/hive-1.1.0-cdh5.7.0.tar.gz  
   * wget http://ftp.ntu.edu.tw/MySQL/Downloads/Connector-J/mysql-connector-java-5.1.46.tar.gz
* 解压：tar -zxvf hive-1.1.0-cdh5.7.0.tar.gz -C ~/app/

* 配置：  
export HIVE_HOME=/home/hadoop/app/hive-1.1.0-cdh5.7.0  
export PATH=$HIVE_HOME/bin:$PATH  

* 本地独立模式
```
# yum install mysql mysql-server          //安装mysql
# service mysqld start
# mysql -u root                           //添加数据库及用户
mysql> create database hive;      
Query OK, 1 row affected (0.00 sec)
mysql> grant all on hive.* to 'hive'@'localhost' identified by 'hive';
Query OK, 0 rows affected (0.00 sec)
mysql> flush privileges;
Query OK, 0 rows affected (0.00 sec)
```

```
$ vim /usr/hadoop/conf/hive-site.xml 
<property>
  <name>javax.jdo.option.ConnectionURL</name>         //所连接的MySQL数据库实例
  <value>jdbc:mysql://localhost:3306/sparksql?createDatabaseIfNotExist=true</value>
  <description>JDBC connect string for a JDBC metastore</description>
</property>
 
<property>
  <name>javax.jdo.option.ConnectionDriverName</name>  //连接的MySQL数据库驱动
  <value>com.mysql.jdbc.Driver</value>
  <description>Driver class name for a JDBC metastore</description>
</property>
 
<property>
  <name>javax.jdo.option.ConnectionUserName</name>   //连接的MySQL数据库用户名
  <value>root</value>
  <description>username to use against metastore database</description>
</property>
 
<property>
  <name>javax.jdo.option.ConnectionPassword</name>    //连接的MySQL数据库密码
  <value>root</value>
  <description>password to use against metastore database</description>
</property>
```

*   拷贝mysql驱动到$HIVE_HOME/lib/  
cp ~/software/mysql-connector-java-5.1.46/mysql-connector-java-5.1.46.jar .
 /home/hadoop/software/mysql-connector-java-5.1.46  

UNIX socket:		/var/lib/mysql/mysql.sock

* 启动hive

/home/hadoop/app/hive-1.1.0-cdh5.7.0/conf









<a id='2'></a>
### [第2章 Spark及其生态圈概述](#0)

 


https://spark.apache.org/
### 2.2. Spark 概述及特点
分布式计算框架
相比hadoop速度快很多
 <img src='https://spark.apache.org/images/logistic-regression.png' >
### 2.2. Spark 产生背景
* mapreduce的局限性
  * 代码繁琐
  * 只能支持map和reduce操作
  * 执行效率低下
  * 不适合迭代多次、交互式、流式的处理
* 框架多样化
  * 批处理（离线）：MapReduce，Hive，Pig
  * 流式处理（实时）：storm，jstom
  * 交互式计算：Impala   
  
> 框架多样化，导致需要多态集群，学习运维成本高

### 2.4. Spark发展历史
<img src='img/Spark发展历史.png'，width='66%'>
### 2.6. Spark 对比 Hadoop
### 2.7. 

<a id='3'></a>
# [第3章 实战环境搭建](#0)

### 3.1 Spark 源码编译
* 下载  wget https://archive.apache.org/dist/spark/spark-2.1.0/spark-2.1.0.tgz
* 解压  解压：tar -zxvf spark-2.1.0.tgz -C ~/app/
* 要求： Building Spark using Maven requires Maven 3.3.9 or newer and Java 7+.
* Maven3.3.9安装包下载 wget https://mirrors.tuna.tsinghua.edu.cn/apache//maven/maven-3/3.3.9/binaries/apache-maven-3.3.9-bin.tar.gz
* 解压： tar -zxvf apache-maven-3.3.9-bin.tar.gz -C ~/app/
* maven内存设置 export MAVEN_OPTS="-Xmx2g -XX:ReservedCodeCacheSize=512m"
* 编译命令
  * ./build/mvn -Pyarn -Phadoop-2.6 -Phive -Phive-thriftserver  -Dhadoop.version=2.6.0-cdh5.7.0 -DskipTests clean package

  * 推荐： ./dev/make-distribution.sh --name 2.6.0-cdh5.7.0  --tgz -Pyarn -Phadoop-2.6 -Phive -Phive-thriftserver -Dhadoop.version=2.6.0-cdh5.7.0  -Pmesos  
  ./dev/make-distribution.sh --name 2.6.0-cdh5.7.0  --tgz -Pyarn -Phadoop-2.6 -Phive -Phive-thriftserver -Dhadoop.version=2.6.0-cdh5.7.0  -Pmesos 
    第二个编译完成后：spark-\$VERSINO-bin-\$NAME 



#### scala 安装配置哦
wgethttps://downloads.lightbend.com/scala/2.11.8/scala-2.11.8.tgz
解压：tar -zxvf scala-2.11.8.tgz -C ~/app/
设置Scala环境变量设置

SCALA_HOME=/opt/scala-2.11.7
PATH=$PATH:$SCALA_HOME/bin
export SCALA_HOME PATH
验证Scala安装

scala -version
Scala code runner version 2.11.7 -- Copyright 2002-2013, LAMP/EPFL

In [ ]:
 //将JDK目录添加到系统环境变量(~/.bash_profile)中 
export MAVEN_HOME=/home/hadoop/app/apache-maven-3.3.9 
export PATH=$MAVEN_HOME/bin:$PATH 
//让配置文件生效 source ~/.bash_profile
//执行mvn，查看版本
mvn -v 
//如果安装成功后，则有如下信息的输出 
Apache Maven 3.3.9 (bb52d8502b132ec0a5a3f4c09453c07478323dc5; 2015-11-10T08:41:47-08:00) 
Maven home: /home/hadoop/app/apache-maven-3.3.9 Java version: 1.7.0_51, vendor: Oracle Corporation 
Java home: /home/hadoop/app/jdk1.7.0_51/jre 
Default locale: zh_CN, platform encoding: UTF-8 
OS name: "linux", version: "2.6.32-358.el6.x86_64", arch: "amd64", family: "unix"
 

坑1：编译报错；
Failed to execute goal on project spark-launcher_2.11: Could not resolve dependencies for project org.apache.spark:spark-launcher_2.11:jar:2.1.0: Failure to find org.apache.hadoop:hadoop-client:jar:2.6.0-cdh5.7.0 in https://repo1.maven.org/maven2 was cached in the local repository, resolution will not be reattempted until the update interval of central has elapsed or updates are forced -> [Help 1]

坑2：虚拟机内存：2-4G
坑3： ./dev/change-scala-version.sh 2.11

https://www.cloudera.com/documentation/enterprise/release-notes/topics/cdh_vd_cdh5_maven_repo.html  
https://spark.apache.org/docs/2.1.0/building-spark.html  
https://blog.csdn.net/wuzhilon88/article/details/73240571

In [ ]:
# 报错，在pom.xml的<id>central</id>下插入

 
<repository>
    <id>cloudera</id>
       <url>https://repository.cloudera.com/artifactory/cloudera-repos/</url>
</repository

<properties>
 
    <hadoop.version>2.2.0</hadoop.version>
    <protobuf.version>2.5.0</protobuf.version>
    <yarn.version>${hadoop.version}</yarn.version>
</properties>


<profile>
  <id>hadoop-2.6</id>
  <properties>
    <hadoop.version>2.6.4</hadoop.version>
    <jets3t.version>0.9.3</jets3t.version>
    <zookeeper.version>3.4.6</zookeeper.version>
    <curator.version>2.6.0</curator.version>
  </properties>
</profile>
